<a href="https://colab.research.google.com/github/sergoMedvedev/BEST_HACK/blob/main/text_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Об используемых инструментах:

TF IDF https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer

DBSCAN https://scikit-learn.org/stable/auto_examples/cluster/plot_dbscan.html#sphx-glr-auto-examples-cluster-plot-dbscan-

KMeans https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

In [ ]:
min_samples = 25
n_clusters = 6

In [ ]:
import pandas as pd
from nltk import download
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import re
import numpy as np
from transformers import pipeline, AutoTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN, KMeans, SpectralClustering
from matplotlib import pyplot as plt

Prepare data and libraries

In [ ]:
df = pd.read_json("drive/MyDrive/dataset_hack.json")
stemmer = SnowballStemmer("russian")
download('stopwords')
stopwords = stopwords.words("russian") + stopwords.words("english") + ["Привет!", "привет"]
tokenizer = AutoTokenizer.from_pretrained('distilroberta-base')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df.head()

,id,message,user_id,channel_id,create_at,update_at,root_id,edit_at,delete_at,is_pinned,original_id
0,146090,Привет! Покупателю 473905 не возвращаются день...,261636,897391,2023-08-29 22:03:17.662,2023-08-29 22:03:17.662,None,0,0,False,
1,146099,Привет! Покупателю 310597 не возвращаются день...,205592,347308,2023-08-29 23:52:18.227,2023-08-29 23:52:18.227,None,0,0,False,
2,146104,"Добрый день, в профиле 528509 не отображается ...",178096,250509,2023-08-30 01:00:56.713,2023-08-30 01:00:56.713,None,0,0,False,
3,146111,Привет! У клиента 804993 несколько вопросов ка...,255493,69518,2023-08-30 02:29:30.042,2023-08-30 02:29:30.042,None,0,0,False,
4,146118,Привет! У клиента 746656 несколько вопросов ка...,660691,52200,2023-08-30 04:15:40.920,2023-08-30 04:15:40.920,dsyhsescphovx9o0jktmy0zt,0,0,False,


In [ ]:
messages = df["message"]

In [ ]:
messages

0       Привет! Покупателю 473905 не возвращаются день...
1       Привет! Покупателю 310597 не возвращаются день...
2       Добрый день, в профиле 528509 не отображается ...
3       Привет! У клиента 804993 несколько вопросов ка...
4       Привет! У клиента 746656 несколько вопросов ка...
                              ...                        
4995    Привет! Пользователь 854633 просит убрать увед...
4996    Привет! Продавец 357850 просит убрать уведомле...
4997    Привет! Была выплачена компенсация №286603, ну...
4998    Привет! У клиента 785331 несколько вопросов ка...
4999    Добрый день, в профиле 27577 не отображается 1...
Name: message, Length: 5000, dtype: object

# Text preprocessing

Lowercase

In [ ]:
messages = [message.lower() for message in messages]

Removing special symbol

In [ ]:
def remove_custom_symbols(words_string:str) -> str:
  return re.sub(r'[^A-Za-zА-Яа-я ]+', "", words_string)

In [ ]:
messages = [remove_custom_symbols(message) for message in messages]

Stemming

In [ ]:
def stem_words(words_string:str) -> str:
  return " ".join([stemmer.stem(word) for word in words_string.split(" ")])

In [ ]:
messages = [stem_words(message) for message in messages]

Removing stopwords

In [ ]:
def remove_stopwords(words_string:str) -> str:
  return " ".join([word for word in words_string.split(" ") if word not in stopwords])

In [ ]:
messages = [remove_stopwords(message) for message in messages]

Preprocessed strings

In [ ]:
messages[:10]

['покупател  возвраща деньг статус завис confirmed ',
 'покупател  возвраща деньг статус завис confirmed ',
 'добр ден профил  отобража  дне возврат подскаж эт включ',
 'клиент  нескольк вопрос касаем доставк dbs rdbs api подсказа ответ ил подскажет ком можн направ подробн тред',
 'клиент  нескольк вопрос касаем доставк dbs rdbs api подсказа ответ ил подскажет ком можн направ подробн тред',
 'покупател  возвраща деньг статус завис refunding',
 'заказ  политик возврат покупател отказа заказ деньг вернул биллинг статус заказ lost помог пожалуйст вернут деньг пользовател',
 'заказ  политик возврат покупател отказа заказ деньг вернул биллинг статус заказ cancelled помог пожалуйст вернут деньг пользовател',
 'клиент  нескольк вопрос касаем доставк dbs rdbs api подсказа ответ ил подскажет ком можн направ подробн тред',
 'покупател  возвраща деньг статус завис confirmed ']

# Getting topics

TF IDF vectorization

In [ ]:
tfidf = TfidfVectorizer()

In [ ]:
tfidf.fit(messages)
transformed = tfidf.transform(messages)

In [ ]:
tfidf.get_feature_names_out()

words_df = pd.DataFrame(transformed[0].T.todense(), index=tfidf.get_feature_names_out(), columns=["TF-IDF"])
words_df = words_df.sort_values('TF-IDF', ascending=False)

Top words by importance

In [ ]:
words_df

,TF-IDF
confirmed,0.517349
завис,0.488510
возвраща,0.488510
покупател,0.300135
деньг,0.300135
...,...
зависл,0.000000
дс,0.000000
доставк,0.000000
добр,0.000000


Clustering

In [ ]:
#db = DBSCAN(eps=0.3, min_samples=min_samples).fit(transformed)

db = KMeans(n_clusters=n_clusters).fit(transformed)

db = SpectralClustering(n_clusters=n_clusters).fit(transformed)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
labels = db.labels_

In [ ]:
labels

array([5, 5, 1, ..., 2, 3, 1], dtype=int32)

In [ ]:
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)

Estimated number of clusters: 6
Estimated number of noise points: 0


In [ ]:
label_series = pd.Series(labels)
freq_table = label_series.value_counts()

In [ ]:
freq_table

0    1846
2    1375
5     686
4     381
1     365
3     347
Name: count, dtype: int64

In [ ]:
df["label"] = db.labels_

In [ ]:
df.head()

,id,message,user_id,channel_id,create_at,update_at,root_id,edit_at,delete_at,is_pinned,original_id,label
0,146090,Привет! Покупателю 473905 не возвращаются день...,261636,897391,2023-08-29 22:03:17.662,2023-08-29 22:03:17.662,None,0,0,False,,5
1,146099,Привет! Покупателю 310597 не возвращаются день...,205592,347308,2023-08-29 23:52:18.227,2023-08-29 23:52:18.227,None,0,0,False,,5
2,146104,"Добрый день, в профиле 528509 не отображается ...",178096,250509,2023-08-30 01:00:56.713,2023-08-30 01:00:56.713,None,0,0,False,,1
3,146111,Привет! У клиента 804993 несколько вопросов ка...,255493,69518,2023-08-30 02:29:30.042,2023-08-30 02:29:30.042,None,0,0,False,,3
4,146118,Привет! У клиента 746656 несколько вопросов ка...,660691,52200,2023-08-30 04:15:40.920,2023-08-30 04:15:40.920,dsyhsescphovx9o0jktmy0zt,0,0,False,,3


In [ ]:
import time

In [ ]:
time = int(time.time())

In [ ]:
df.to_csv(f"drive/MyDrive/dataset_hack_with_labels_min_{time}.csv")